In [ ]:
from wq_api import *

session = sign_in(username,password)

In [ ]:
# 参数,alpha表达式见后
setting = {
    'region': 'GLB', 
    'universe': 'MINVOL1M', 
    'delay': 1,
    'decay': 4,
    'neutralization':'SUBINDUSTRY'} 

dataset_id_ls = [{'id':'analyst9','backfill':False},
                 {'id':'analyst69','backfill':False},
                 {'id':'analyst44','backfill':False},
                 ]

# 只要vector还是只要matrix,目前只支持处理一种数据类型
vec_only = False

dir_path = './727'

# 仅作为记录，并不作为操作，alpha表达式仍需在之后修改
template = 'piece_1 = group_mean(vec_stddev({}) , 1 , subindustry) - vec_stddev({});ts_mean(piece_1, 60)'


In [ ]:
import os

record_file = 'record.csv'

rows = []
for item in dataset_id_ls:
    row = {'template':template, 'vec_only':vec_only, 'dir_path':dir_path, **item, **setting}
    rows.append(row)
combined_df = pd.DataFrame(rows)

if os.path.exists(record_file):
    existing_df = pd.read_csv(record_file)
    combined_df = pd.concat([existing_df, combined_df], ignore_index=True)
else:
    combined_df.to_csv(record_file, index=False)

combined_df.to_csv(record_file, index=False)

In [ ]:
# data is restored in {index}_refine_1.csv, where index is generated by order

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

index = 0
while True:
    to_path = f'{dir_path}/{index}_data_1.csv'
    if not os.path.exists(to_path):
        break
    index += 1

print(f'Next available file path: {to_path}')

In [ ]:
import pandas as pd
df = pd.DataFrame()

for dataset in dataset_id_ls:
    dataset_id = dataset['id']
    backfill = dataset['backfill']
    
    datafields_df = get_datafields(session, dataset_id=dataset_id, region=setting['region'], universe=setting['universe'], delay=setting['delay'])
    if vec_only:
        datafields_df = datafields_df[datafields_df['type'] == "VECTOR"][['id','dataset']]
    else:
        datafields_df = datafields_df[datafields_df['type'] == "MATRIX"][['id','dataset']]

    datafields_df['backfill'] = backfill
    
    df = pd.concat([df, datafields_df])
df.reset_index(drop=True, inplace=True)
df

In [ ]:
for index, row in df.iterrows():
    if(row['backfill']==True):
        df.loc[index, 'regular'] = "piece_1 = group_mean(vec_stddev({}) , 1 , subindustry) - vec_stddev({});ts_mean(piece_1, 60)" \
                          .format(row['id'],row['id'])
    else:
        df.loc[index, 'regular'] = "piece_1 = group_mean(vec_stddev({}) , 1 , subindustry) - vec_stddev({});ts_mean(piece_1, 60)" \
                          .format(row['id'],row['id'])

In [ ]:
# make test alpha list
alpha_list=[]
for index, row in df.iterrows():
    alpha_list.append(
        {
            'type': 'REGULAR',
            'settings': {
                'instrumentType': 'EQUITY',
                'region': 'GLB',
                'universe': setting['universe'],
                'delay': setting['delay'],
                'decay': setting['decay'],
                'neutralization': setting['neutralization'],
                'truncation': 0.08,
                'pasteurization': 'ON',
                'unitHandling': 'VERIFY',
                'nanHandling': 'ON',
                'language': 'FASTEXPR',
                'visualization': False,
            },
            'regular': row['regular']
        })
alpha_list

In [ ]:
alpha_df = pd.DataFrame(alpha_list)
settings_df = alpha_df['settings'].apply(pd.Series)
alpha_df = pd.concat([alpha_df.drop(columns=['settings']), settings_df], axis=1)
alpha_df.set_index(['regular'],inplace=True)
alpha_df

In [ ]:
df.set_index(['regular'],inplace=True)
# 将展开的 settings 列与其他列合并
df = pd.concat([df, alpha_df], axis=1)

print(df)

df.to_csv(to_path)